In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device=('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data_set=np.loadtxt('data/pima-indians-diabetes3.csv', delimiter=',', skiprows=1) #numpy에서 첫줄 들어오니까 한줄 skip 하고 가져오기

In [4]:
class Pima(nn.Module):
    def __init__(self):
        super(Pima, self).__init__()
        self.hidden_linear=nn.Linear(8, 12)
        self.hidden_activation=nn.ReLU()
        self.hidden_linear2=nn.Linear(12, 8)
        self.hidden_activation2=nn.ReLU()
        self.output_linear=nn.Linear(8,1)
        self.output_activation=nn.Sigmoid()
        
    def forward(self, x):
        x=self.hidden_linear(x)
        x=self.hidden_activation(x)
        x=self.hidden_linear2(x)
        x=self.hidden_activation2(x)
        x=self.output_linear(x)
        x=self.output_activation(x)
        
        return x
    
    def predict(self, x):
        pred=self.forward(x)
        if pred >= 0.5:
            return 1
        else: 
            return 0

In [5]:
model=Pima().to(device) #bias=False (bias 안씀)
model

Pima(
  (hidden_linear): Linear(in_features=8, out_features=12, bias=True)
  (hidden_activation): ReLU()
  (hidden_linear2): Linear(in_features=12, out_features=8, bias=True)
  (hidden_activation2): ReLU()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
  (output_activation): Sigmoid()
)

In [6]:
param_gen=model.named_parameters()
param_gen

<generator object Module.named_parameters at 0x000001C9A137D970>

In [7]:
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', ' x '.join(str(x) for x in list(param.size())[::-1]), '=', num_param)
            else:
                print(name, ':', num_param)
                print('-' * 40)
            total_param += num_param
    print('total:', total_param)

In [8]:
count_parameters(model)

hidden_linear.weight : 8 x 12 = 96
hidden_linear.bias : 12
----------------------------------------
hidden_linear2.weight : 12 x 8 = 96
hidden_linear2.bias : 8
----------------------------------------
output_linear.weight : 8 x 1 = 8
output_linear.bias : 1
----------------------------------------
total: 221


In [9]:
data_set=torch.from_numpy(data_set)
X=data_set[:, :-1]
y=data_set[:, -1]

ds=TensorDataset(X, y)
dataloader= DataLoader(ds, batch_size=5)
optimizer=optim.Adam(model.parameters())

n_epochs=100
loss_fn=nn.BCELoss()

for epoch in range(n_epochs):
    for data, label in dataloader:
        out=model(data.type(torch.FloatTensor).to(device)) #forward propagation
        loss=loss_fn(out, label.type(torch.FloatTensor).unsqueeze(1).to(device)) #unsqueeze(=expanddim 차원 늘리기) ; 현재는 0,1번째 차원 사이 차원 증가 #calculate loss (배치 내 손실 총합)
        #torch.nn.BCELoss는 PyTorch 라이브러리에서 제공하는 클래스로, 이진 분류(Binary Classification) 문제에서 사용되는 이진 교차 엔트로피(Binary Cross Entropy) 손실 함수를 구현한 것입니다. 
        # BCELoss는 예측값과 타겟값 사이의 교차 엔트로피를 계산하여 손실 값을 얻는데 사용됩니다.
        loss.backward() #개별 파라미터 기울기 계산
        optimizer.step() #개별 파라미터 갱신
        optimizer.zero_grad() #optimizer가 계산해서 저장해 놓은 기울기값 clear #"Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문"에 우리는 항상 backpropagation을 하기전에 gradients를 zero로 만들어주고 시작을 해야합니다.
    print("Epoch: %d, Loss: %.4f" %(epoch, float(loss)))

Epoch: 0, Loss: 0.4682
Epoch: 1, Loss: 0.4561
Epoch: 2, Loss: 0.4643
Epoch: 3, Loss: 0.4693
Epoch: 4, Loss: 0.4698
Epoch: 5, Loss: 0.4672
Epoch: 6, Loss: 0.4574
Epoch: 7, Loss: 0.4621
Epoch: 8, Loss: 0.4626
Epoch: 9, Loss: 0.4619
Epoch: 10, Loss: 0.4622
Epoch: 11, Loss: 0.4637
Epoch: 12, Loss: 0.4627
Epoch: 13, Loss: 0.4579
Epoch: 14, Loss: 0.4573
Epoch: 15, Loss: 0.4495
Epoch: 16, Loss: 0.4429
Epoch: 17, Loss: 0.4408
Epoch: 18, Loss: 0.4403
Epoch: 19, Loss: 0.4425
Epoch: 20, Loss: 0.4463
Epoch: 21, Loss: 0.4342
Epoch: 22, Loss: 0.4294
Epoch: 23, Loss: 0.4315
Epoch: 24, Loss: 0.4391
Epoch: 25, Loss: 0.4119
Epoch: 26, Loss: 0.4009
Epoch: 27, Loss: 0.4065
Epoch: 28, Loss: 0.3663
Epoch: 29, Loss: 0.3604
Epoch: 30, Loss: 0.3727
Epoch: 31, Loss: 0.3508
Epoch: 32, Loss: 0.3449
Epoch: 33, Loss: 0.3534
Epoch: 34, Loss: 0.3458
Epoch: 35, Loss: 0.3471
Epoch: 36, Loss: 0.3486
Epoch: 37, Loss: 0.3525
Epoch: 38, Loss: 0.3373
Epoch: 39, Loss: 0.3254
Epoch: 40, Loss: 0.3247
Epoch: 41, Loss: 0.3373
Ep

In [10]:
train_loader = DataLoader(ds)    # default batch는 1
correct = 0
total = 0
with torch.no_grad():  # torch는 기울기 계산을 위해 순전파시 grad를 작성하는데 테스트시에 필요 없으므로 no_grad 해준다.
    for data, label in train_loader:
        data = data.type(torch.FloatTensor)
        predicted = model.predict(data.to(device))
        target = label[0].item()    # .item: tensor에서 제공하는 스칼라 값을 뽑아주는 함수
        total += 1
        correct += 1 if (predicted ==target) else 0
print('Accuracy: %.4f' % (correct/ total))

Accuracy: 0.7786
